# Medicare Fraud Detection

### Data 698: Data Science Research Project - Spring 2019

### Student: Chunhui Zhu

Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2016.csv: https://data.cms.gov/Medicare-Physician-Supplier/Medicare-Provider-Utilization-and-Payment-Data-Phy/utc4-f9xp

LEIE2019.csv: https://oig.hhs.gov/exclusions/exclusions_list.asp

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib as plt
import pickle

import matplotlib.pyplot as plt
from pandas import Series, DataFrame

## Remove Colinearity 

#I used the searching result of unique NPI finding in a specifit year in previous steps. 
#load Fraud unique NPI from pickle, example for 2016 yr: "fraud_df16_pickle"
#close pickle
#function returns pd df for fraud_NPI in that year
def loadfraudNPI (year):
    y=str(year)
    fraud_pkl=open("fraud_df"+str(y[2:4])+"_pickle","rb")
    fraud_NPI=pickle.load(fraud_pkl)
    print(fraud_NPI.head(5))
    print("The number of fraud NPI found in " + y +" is " + str(len(fraud_NPI)) + ".")
    fraud_pkl.close()
    return (fraud_NPI)

In [2]:
#example pickle name conver2016_pickle
def loadconvtdf (year):
    df_pkl=open("convt"+str(year)+"_pickle","rb")
    df=pickle.load(df_pkl)
    print("finished load for convt"+str(year)+"_pickle.")
    df_pkl.close()
    return (df)

In [3]:
convt2016df=loadconvtdf(2016)
convt2015df=loadconvtdf(2015)
convt2014df=loadconvtdf(2014)
convt2013df=loadconvtdf(2013)
convt2012df=loadconvtdf(2012)

finished load for convt2016_pickle.
finished load for convt2015_pickle.
finished load for convt2014_pickle.
finished load for convt2013_pickle.
finished load for convt2012_pickle.


In [4]:
#made sure it is the right data set from last process
convt2015df.head()

,npi,type,hcpcs_c,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,avg_allowed_amt,avg_payment_amt,avg_submitted_chrg_amt,city,state,Fraud
1,1003000126,0.0,99217,23.0,23.0,23.0,72.680000,54.502609,328.000000,3,2,0
2,1003000126,0.0,99219,18.0,18.0,18.0,135.850000,100.958889,614.000000,3,2,0
3,1003000126,0.0,99221,59.0,58.0,59.0,101.365085,79.469661,333.288136,3,2,0
4,1003000126,0.0,99222,132.0,130.0,132.0,139.010455,107.933409,356.492424,3,2,0
5,1003000126,0.0,99223,220.0,215.0,220.0,205.185955,157.908955,631.186364,3,2,0


In [5]:
from pandas import *
import numpy as np
from scipy.stats.stats import pearsonr
import itertools

## Step 1: Calculating pairwise correlation among all columns

### since each year data are a lot and dupilicate rows, I choosed 2012 and 2015 in observation. 

In [6]:
df_list=[convt2015df,convt2012df]
df_convt = pd.concat(df_list)

#remove category data 
num_df_convt=df_convt.drop(columns=["npi","hcpcs_c"])

In [7]:
correlations = {}
columns = num_df_convt.columns.tolist()

for col_a, col_b in itertools.combinations(columns, 2):
    correlations[col_a + '__' + col_b] = pearsonr(num_df_convt.loc[:, col_a], num_df_convt.loc[:, col_b])

result = DataFrame.from_dict(correlations, orient='index')
result.columns = ['PCC', 'p-value']

print(result.sort_index())


                                                PCC        p-value
avg_allowed_amt__Fraud                     0.000172   4.724211e-01
avg_allowed_amt__avg_payment_amt           0.998607   0.000000e+00
avg_allowed_amt__avg_submitted_chrg_amt    0.752158   0.000000e+00
avg_allowed_amt__city                     -0.016892   0.000000e+00
avg_allowed_amt__state                    -0.030493   0.000000e+00
avg_payment_amt__Fraud                     0.000110   6.457250e-01
avg_payment_amt__avg_submitted_chrg_amt    0.751998   0.000000e+00
avg_payment_amt__city                     -0.016705   0.000000e+00
avg_payment_amt__state                    -0.031322   0.000000e+00
avg_submitted_chrg_amt__Fraud             -0.002300   7.802266e-22
avg_submitted_chrg_amt__city              -0.026393   0.000000e+00
avg_submitted_chrg_amt__state             -0.021499   0.000000e+00
bene_day_srvc_cnt__Fraud                   0.002345   1.268145e-22
bene_day_srvc_cnt__avg_allowed_amt        -0.012051   0.000000

## Step 2: Drop variables

From PCC and P-value for each pair variables, it showed some varibles having strong correlation since they are based on same factor when they were created. If they are all in a model, it brings Colinearity which causes the high covariance and untrusted the accuracy in R-squar.

I reduced some varible in following steps based on the higher absolute PCC with Fraud. 

avg_allowed_amt,  avg_payment_amt,  avg_submitted_chrg_amt  => keep: avg_submitted_chrg_amt

bene_unique_cnt,  bene_day_srvc_cnt => Keep: bene_day_srvc_cnt 

In [8]:
#example pickle name conver2016_pickle
def saveinpkl (df,year):
    drop_columns_df=df.drop(columns=['avg_allowed_amt','avg_payment_amt','bene_unique_cnt'])
    df_pkl=open("convt_dropcolumns_"+str(year)+"_pickle","wb")
    pickle.dump(drop_columns_df,df_pkl)
    print("saved convt_dropcolumns_"+str(year)+"_pickle.")
    df_pkl.close()
    
    return

In [9]:
saveinpkl (convt2016df, 2016)
saveinpkl (convt2015df, 2015)
saveinpkl (convt2014df, 2014)
saveinpkl (convt2013df, 2013)
saveinpkl (convt2012df, 2012)

saved convt_dropcolumns_2016_pickle.
saved convt_dropcolumns_2015_pickle.
saved convt_dropcolumns_2014_pickle.
saved convt_dropcolumns_2013_pickle.
saved convt_dropcolumns_2012_pickle.


## References:

Variance Inflation Factor (VIF) Explained
https://etav.github.io/python/vif_factor_python.html